# Final Project (ECON570)
* Instructor: Dr. Ida Johnsson
* Authors: Seongmoon CHO, James GROSS, and Jaehyun HA
* Date: 05/06/2022

# Data Cleaning

In [54]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt

In [55]:
df = pd.read_csv('train.csv') #read csv file
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [56]:
# df.info() # check data quality; there are several missing data; there are also numerical values and categorical values in the data

In [57]:
# variables with null values
df.isnull().sum()[df.isnull().sum() > 0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [58]:
# fill 0 for null values in numerical variables
temp = ['LotFrontage', 'MasVnrArea'] # we did not fill 0 for null values in GarageYrBlt because it is a 'year' value
for i in temp:
    df[i].fillna(0, inplace=True)

In [59]:
# for YearBuilt and YearRemodAdd -> calculate years since each time period to sold year
df['YearBuilt'] = df['YrSold'] - df['YearBuilt'] 
df['YearRemodAdd'] = df['YrSold'] - df['YearRemodAdd'] 

In [60]:
# code categorical variables to numerical variables
df['LandContour'].value_counts()

Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64

In [61]:
# Street (1 if Pave) 
df['Street'] = df.Street.replace({"Grvl" : 0, "Pave" : 1})

In [62]:
# Alley (1 if has alley access regardless of type)
df['Alley'] = df.Alley.replace({"NA" : 0, "Grvl" : 1, "Pave" : 1})
df['Alley'].fillna(0, inplace = True)

In [63]:
# LotShape (1 if the shape of property is regular)
df['LotShape'] = df.LotShape.replace({"IR1":0, "IR2":0, "IR3":0, "Reg":1})

In [64]:
# LandContour (1 if near flat)
df['LandContour'] = df.LandContour.replace({"Bnk":0, "HLS":0, "Low":0, "Lvl":1})

In [65]:
# Utilities (1 if all public utilities)
df['Utilities'] = df.Utilities.replace({"NoSeWa":0, "AllPub":1})

In [66]:
# LotConfig (make dummy variables and append to df)
df = pd.concat([df, pd.get_dummies(df['LotConfig'])], axis=1)

In [67]:
# LandSlope (1 if gentle slope)
df['LandSlope'] = df.LandSlope.replace({"Gtl":1, "Mod":0, "Sev":0})

In [68]:
# Neighborhood (make dummy variables and append to df)
df = pd.concat([df, pd.get_dummies(df['Neighborhood'])], axis=1)

In [69]:
# BldgType (there are five categories, but we reduced it to three and created as dummy variables)
df['BldgType'] = df.BldgType.replace({"1Fam": "1Fam", "2fmCon": "2Fam", "Duplex": "2Fam", "TwnhsE": "Twnhs", "TwnhsI": "Twnhs"})
df = pd.concat([df, pd.get_dummies(df['BldgType'])], axis=1)

In [70]:
# HouseStyle (make dummy variables and append to df)
df = pd.concat([df, pd.get_dummies(df['HouseStyle'])], axis=1)

In [71]:
# Roofstyle (there are six categories, but we reduced it to three and created as dummy variables)
df["RoofStyle"] = df.RoofStyle.replace({"Flat": "Others", "Gambrel": "Others", "Mansard": "Others", "Shed": "Others"})
df = pd.concat([df, pd.get_dummies(df['RoofStyle'])], axis=1)

In [72]:
# RoofMatl (1 if Standard Shingle)
df['RoofMatl'] = df.RoofMatl.replace({"CompShg":1, "Tar&Grv":0, "WdShngl":0, "WdShake":0, "Roll":0, "Membran":0, "ClyTile":0, "Metal":0})

In [73]:
# MasVnrType (fill null values with None and created as dummy variables)
df["MasVnrType"].fillna("None", inplace=True)
df = pd.concat([df, pd.get_dummies(df['MasVnrType'])], axis=1)

In [74]:
# ExterQual 
df['ExterQual'] = df.ExterQual.replace({'None': 0, 'NA': 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5})

In [75]:
# ExterCond 
df['ExterCond'] = df.ExterCond.replace({'None': 0, 'NA': 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5})

In [76]:
# Basement related variables 
df['BsmtQual'] = df.BsmtQual.replace({'None': 0, 'NA': 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5})
df['BsmtCond'] = df.BsmtCond.replace({'None': 0, 'NA': 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5})
df['BsmtExposure'] = df.BsmtExposure.replace({'None': 0, 'NA': 0, 'No' : 0, 'Mn' : 1, 'Av' : 2, 'Gd' : 3})
df['BsmtFinType1'] = df.BsmtFinType1.replace({'None': 0, 'NA': 0, 'Unf' : 1, 'LwQ' : 2, 'Rec' : 3, 'BLQ' : 4, 'ALQ' : 5, 'GLQ' : 6})
df['BsmtFinType2'] = df.BsmtFinType2.replace({'None': 0, 'NA': 0, 'Unf' : 1, 'LwQ' : 2, 'Rec' : 3, 'BLQ' : 4, 'ALQ' : 5, 'GLQ' : 6})

In [77]:
# Heating (1 if GasA)
df['Heating'] = df.Heating.replace({"GasA":1, "Floor":0, "GasW":0, "Grav":0, "OthW":0, "Wall":0})
df['HeatingQC'] = df.HeatingQC.replace({'None': 0, 'NA': 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5})

In [78]:
# CentralAir (1 if Y)
df['CentralAir'] = df.Heating.replace({"Y":1, "N":0})

In [79]:
# Electrical (1 if SBrkr)
df['Electrical'] = df.Electrical.replace({"SBrkr":1, "FuseA":0, "FuseF":0, "FuseP":0, "Mix":0})

In [80]:
# KitchenQual
df['KitchenQual'] = df.KitchenQual.replace({'None': 0, 'NA': 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5})

In [81]:
# FireplaceQu
df['FireplaceQu'] = df.FireplaceQu.replace({'None': 0, 'NA': 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5})

In [82]:
# Garage related variables
df['GarageFinish'] = df.GarageFinish.replace({'None': 0, 'NA': 0, 'Unf' : 1, 'RFn' : 2, 'Fin' : 3})
df['GarageQual'] = df.GarageQual.replace({'None': 0, 'NA': 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5})
df['GarageCond'] = df.GarageCond.replace({'None': 0, 'NA': 0, 'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5})

In [83]:
# PavedDrive
df['PavedDrive'] = df.PavedDrive.replace({'N': 0, 'P': 1, 'Y' : 2})

In [84]:
# Fence
df['Fence'] = df.Fence.replace({'NA': 0, 'MnWw' : 1, 'GdWo' : 2, 'MnPrv' : 3, 'GdPrv' : 4})

In [85]:
# YrSold (make dummy variables and append to df)
df = pd.concat([df, pd.get_dummies(df['YrSold']).rename(columns=lambda x:'Y_' +str(x))], axis=1)

In [86]:
# SaleCondition (1 if Normal)
df['SaleCondition'] = df.SaleCondition.replace({'Normal': 1, 'Abnorml': 0, 'AdjLand': 0, 'Alloca': 0, 'Family': 0, 'Partial': 0})

In [87]:
features=['Id', 'SalePrice', 'Neighborhood', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LandSlope',
          'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofMatl', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
          'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', 
          '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
          'Fireplaces', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 
          'ScreenPorch', 'PoolArea', 'SaleCondition', 'Corner', 'CulDSac', 'FR2', 'FR3', 'Inside', 'Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 
          'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NPkVill', 'NWAmes', 'NoRidge', 'NridgHt', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 
          'StoneBr', 'Timber', 'Veenker', '1Fam', '2Fam', 'Twnhs', '1.5Fin', '1.5Unf', '1Story', '2.5Fin', '2.5Unf', '2Story', 'SFoyer', 'SLvl', 'Gable', 'Hip', 'Others', 'BrkCmn', 
          'BrkFace', 'None', 'Stone', 'Y_2006', 'Y_2007', 'Y_2008', 'Y_2009', 'Y_2010']     

df = df[features].copy()

In [88]:
df.isnull().sum()[df.isnull().sum() > 0]

BsmtQual        37
BsmtCond        37
BsmtExposure    38
BsmtFinType1    37
BsmtFinType2    38
Electrical       1
GarageFinish    81
GarageQual      81
GarageCond      81
dtype: int64

In [89]:
#We filled 0 for the null value in some varaibles. They are not simply null values, but 
df.fillna(0, inplace=True)

In [43]:
df.to_csv("data_cleaned.csv", index=False)